## Import

In [6]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import csv

ModuleNotFoundError: No module named 'pyomo.environ'; 'pyomo' is not a package

## Define Data

In [2]:
def getSetFromPath(path):
    data = set()
    with open(path, 'r') as csv_file:
        res = csv.DictReader(csv_file)
        for r in res:
            data.add(r['id'])
        return data

studios = getSetFromPath('../data/studios.csv')
timeslots = getSetFromPath('../data/timeslots.csv')
days = {'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'}
programs = getSetFromPath('../data/programs.csv')
coaches = getSetFromPath('../data/coaches.csv')

q = {}
with open('../data/constraints.csv', 'r') as csv_file:
    data = csv.DictReader(csv_file)
    # Create qualification dictionary
    for c in coaches:
        for p in programs:
            q[(p, c)] = 0
    for d in data:
        for p in d['programs'].split(','):
            if p in programs:
                q[(p, d['coach'])] = 1

p = {}
for s in studios:
    for t in timeslots:
        for d in days:
            for pr in programs:
                for c in coaches:
                    p[(s, t, d, pr, c)] = 1

## Implement

In [3]:
# Model
model = pe.ConcreteModel()

# Sets
model.studios = pe.Set(initialize=studios)
model.timeslots = pe.Set(initialize=timeslots)
model.days = pe.Set(initialize=days)
model.programs = pe.Set(initialize=programs)
model.coaches = pe.Set(initialize=coaches)

# Parameters
model.q = pe.Param(model.programs, model.coaches, initialize=q, default=-1000)
model.p = pe.Param(model.studios, model.timeslots, model.days, model.programs, model.coaches, initialize=p, default=-1000)

# Decision variables
model.x = pe.Var(model.studios, model.timeslots, model.days, model.programs, model.coaches, domain=pe.Boolean)

# Objective function
expr = sum(model.p[s, t, d, p, c] * model.x[s, t, d, p, c]
        for s in model.studios 
        for t in model.timeslots
        for d in model.days
        for p in model.programs
        for c in model.coaches)
model.objective = pe.Objective(sense=pe.maximize, expr=expr)

# Constraint 1: Concurrency
model.concurrency = pe.ConstraintList()
for s in model.studios:
    for t in model.timeslots:
        for d in model.days:
            lhs = sum(model.x[s, t, d, p, c] for p in model.programs for c in model.coaches)
            rhs = 1
            model.concurrency.add(lhs == rhs)

# Constraint 2: Coach Qualification
model.coach_qualifications = pe.ConstraintList()
for p in model.programs:
    for c in model.coaches:
        if (q[p, c] == 0):
            lhs = sum(model.x[s, t, d, p, c] for s in model.studios for t in model.timeslots for d in model.days)
            rhs = 0
            model.coach_qualifications.add(lhs == rhs)

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
data source (type: set).  This WILL potentially lead to nondeterministic
behavior in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


## Solve and Postprocess

In [4]:
# Execute solver
solver = po.SolverFactory('glpk', executable='/opt/homebrew/bin/glpsol')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmpu7zrl2uo.glpk.raw
 --wglp /var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmpkauycn08.glpk.glp
 --cpxlp /var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmp1z0_pnkg.pyomo.lp
Reading problem data from '/var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmp1z0_pnkg.pyomo.lp'...
/var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmp1z0_pnkg.pyomo.lp:3808: warning: lower bound of variable 'x2' redefined
/var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmp1z0_pnkg.pyomo.lp:3808: warning: upper bound of variable 'x2' redefined
111 rows, 945 columns, 1575 non-zeros
945 integer variables, all of which are binary
4753 lines were read
Writing problem data to '/var/folders/7t/552q474s3n7fgqms9j53t0l40000gq/T/tmpkauycn08.glpk.glp'...
3684 lines were written
GLPK Integer Optimizer 5.0
111 rows, 945 columns, 1575 non-zeros
945 integer variables, all of which are binary

In [5]:
# Extract results
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(model.x, names=['s', 't', 'd', 'p', 'c']))
df['x'] = [pe.value(model.x[key]) for key in df.index]
df['p'] = [model.p[key] for key in df.index]

# Print schedule of each studio (day by timeslot)
for studio in studios:
    d_by_t = pd.DataFrame.from_dict({d: {t: 'No schedule' for t in timeslots} for d in days}, orient='index')
    for (s, t, d, p, c), values in df.iterrows():
        if studio == s and values.x == 1:
            d_by_t.loc[(d, t)] = f'{p}-{c}'
    print(studio)
    print(d_by_t.to_markdown())

s2
|           | t4    | t5    | t3    | t2    | t1    |
|:----------|:------|:------|:------|:------|:------|
| friday    | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| saturday  | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| wednesday | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| thursday  | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| sunday    | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| monday    | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| tuesday   | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
s3
|           | t4    | t5    | t3    | t2    | t1    |
|:----------|:------|:------|:------|:------|:------|
| friday    | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| saturday  | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| wednesday | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| thursday  | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| sunday    | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| monday    | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
| tuesday   | p2-c1 | p2-c1 | p2-c1 | p2-c1 | p2-c1 |
s1
|           | t4   